In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
import time

# 1. Organize the dataset directory path
dataset_path = 'E:/Harini/SLIIT/4 year/RP/images'  # Replace with your dataset path

# Function to extract RGB average values from the image
def extract_rgb_features(img_path):
    # Load image and resize to 224x224 for CNN
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)  # Convert image to numpy array
    
    # Normalize pixel values to [0, 1]
    img_array = img_array / 255.0
    
    # Calculate the average RGB values
    avg_rgb = np.mean(img_array, axis=(0, 1))  # Average across width and height
    return avg_rgb

# 2. Preprocessing function for CNN
def prepare_image(img_path):
    # Load image and resize it to 224x224 for CNN
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# 3. Set up the ImageDataGenerator for training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load images from the dataset directory (subfolders are used as class labels)
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    shuffle=True
)

# 4. Build the custom CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(4, activation='softmax'))  # 4 classes: not_mature, partially_mature, mature, rotten

# 5. Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Custom data generator for feeding RGB features (this step is optional, use ImageDataGenerator if you only need to feed images)
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, batch_size, target_size, class_mode):
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size
        self.class_mode = class_mode
        self.classes = os.listdir(directory)
        self.class_indices = {c: idx for idx, c in enumerate(self.classes)}
        self.filenames = []
        
        # Gather all image filenames
        for class_name in self.classes:
            class_dir = os.path.join(directory, class_name)
            for filename in os.listdir(class_dir):
                self.filenames.append((class_name, filename))
        
    def __len__(self):
        return int(np.floor(len(self.filenames) / self.batch_size))
    
    def __getitem__(self, index):
        batch_filenames = self.filenames[index * self.batch_size: (index + 1) * self.batch_size]
        
        # Prepare the batch
        images = []
        rgb_features = []
        labels = []
        
        for class_name, filename in batch_filenames:
            img_path = os.path.join(self.directory, class_name, filename)
            rgb_feature = extract_rgb_features(img_path)
            img_array = prepare_image(img_path)
            
            images.append(img_array)
            rgb_features.append(rgb_feature)
            labels.append(self.class_indices[class_name])
        
        # Convert lists to numpy arrays
        images = np.vstack(images)
        rgb_features = np.vstack(rgb_features)
        labels = to_categorical(labels, num_classes=4)
        
        return images, labels

# 7. Train the model using the custom generator
epochs = 10
batch_size = 32

start_time = time.time()
train_generator = CustomDataGenerator(dataset_path, batch_size=batch_size, target_size=(224, 224), class_mode='categorical')
history = model.fit(train_generator, epochs=epochs)

# 8. Save the model
model.save('papaya_maturity_model_with_rgb_cnn.h5')
print("Model training complete and saved!")

# 9. Get the model version and training details
model_version = model.name
num_classes = len(train_generator.classes)
dataset_size = len(train_generator.filenames)
input_size = (224, 224, 3)
avg_accuracy = np.mean(history.history['accuracy'])
avg_loss = np.mean(history.history['loss'])
avg_confidence = np.mean(history.history['accuracy']) * 100

# Calculate batch processing time
batch_processing_time = (time.time() - start_time) / (len(train_generator) * epochs)

# Display the results
print(f"Model Version: {model_version}")
print(f"Number of Classes: {num_classes}")
print(f"Dataset Size: {dataset_size} images")
print(f"Number of Epochs: {epochs}")
print(f"Batch Size: {batch_size}")
print(f"Input Size: {input_size}")
print(f"Average Confidence: {avg_confidence:.2f}%")
print(f"Average Accuracy: {avg_accuracy * 100:.2f}%")
print(f"Average Loss: {avg_loss:.4f}")
print(f"Batch Processing Time (seconds per batch): {batch_processing_time:.4f}")

# Provide the path to your test image
test_image_path = 'E:/Harini/SLIIT/4 year/RP/images/mature/Mature_003.jpg'  # Replace with the path to the test image

# Preprocess the image
test_image = prepare_image(test_image_path)  # Use the correct function name

# Predict the class
prediction = model.predict(test_image)

# Get the predicted class index (highest probability)
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Mapping of class indices to class labels
class_labels = ['mature', 'not_mature', 'partially_mature', 'rotten']

# Get the predicted class label
predicted_class = class_labels[predicted_class_index]

# Display the prediction result
print(f'The model predicts that the papaya is: {predicted_class}')


Found 1194 images belonging to 4 classes.


C:\Users\Harini\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Harini\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
